<a href="https://colab.research.google.com/github/mohanrajmit/Human-Action-Classification-/blob/master/3dcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget https://storage.googleapis.com/thumos14_files/UCF101_videos.zip

--2022-09-30 13:39:00--  https://storage.googleapis.com/thumos14_files/UCF101_videos.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 74.125.24.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6960400115 (6.5G) [binary/octet-stream]
Saving to: ‘UCF101_videos.zip’

UCF101_videos.zip   100%[===================>]   6.48G  20.4MB/s    in 5m 28s  

2022-09-30 13:44:29 (20.2 MB/s) - ‘UCF101_videos.zip’ saved [6960400115/6960400115]



In [4]:
!unzip /content/UCF101_videos.zip


  inflating: UCF101/v_Punch_g19_c04.avi  
  inflating: UCF101/v_Punch_g19_c05.avi  
  inflating: UCF101/v_Punch_g19_c06.avi  
  inflating: UCF101/v_Punch_g19_c07.avi  
  inflating: UCF101/v_Punch_g20_c01.avi  
  inflating: UCF101/v_Punch_g20_c02.avi  
  inflating: UCF101/v_Punch_g20_c03.avi  
  inflating: UCF101/v_Punch_g20_c04.avi  
  inflating: UCF101/v_Punch_g21_c01.avi  
  inflating: UCF101/v_Punch_g21_c02.avi  
  inflating: UCF101/v_Punch_g21_c03.avi  
  inflating: UCF101/v_Punch_g21_c04.avi  
  inflating: UCF101/v_Punch_g21_c05.avi  
  inflating: UCF101/v_Punch_g21_c06.avi  
  inflating: UCF101/v_Punch_g21_c07.avi  
  inflating: UCF101/v_Punch_g22_c01.avi  
  inflating: UCF101/v_Punch_g22_c02.avi  
  inflating: UCF101/v_Punch_g22_c03.avi  
  inflating: UCF101/v_Punch_g22_c04.avi  
  inflating: UCF101/v_Punch_g22_c05.avi  
  inflating: UCF101/v_Punch_g22_c06.avi  
  inflating: UCF101/v_Punch_g22_c07.avi  
  inflating: UCF101/v_Punch_g23_c01.avi  
  inflating: UCF101/v_Punch_g23_c

In [5]:
import os
files = os.listdir("/content/UCF101")
X = []
labels = []
labellist = []

In [6]:
files

['v_ApplyEyeMakeup_g19_c01.avi',
 'v_PoleVault_g01_c02.avi',
 'v_BandMarching_g20_c06.avi',
 'v_PommelHorse_g04_c05.avi',
 'v_HighJump_g18_c04.avi',
 'v_Billiards_g24_c06.avi',
 'v_CuttingInKitchen_g09_c01.avi',
 'v_BrushingTeeth_g18_c06.avi',
 'v_Diving_g18_c03.avi',
 'v_Basketball_g09_c03.avi',
 'v_BandMarching_g02_c02.avi',
 'v_ApplyEyeMakeup_g18_c01.avi',
 'v_BandMarching_g10_c05.avi',
 'v_JumpingJack_g10_c02.avi',
 'v_Basketball_g11_c05.avi',
 'v_CleanAndJerk_g09_c01.avi',
 'v_PlayingGuitar_g13_c06.avi',
 'v_Kayaking_g03_c03.avi',
 'v_BabyCrawling_g02_c03.avi',
 'v_HandstandWalking_g15_c03.avi',
 'v_BrushingTeeth_g20_c03.avi',
 'v_BasketballDunk_g17_c07.avi',
 'v_RockClimbingIndoor_g06_c03.avi',
 'v_LongJump_g13_c03.avi',
 'v_GolfSwing_g22_c04.avi',
 'v_ApplyEyeMakeup_g24_c02.avi',
 'v_PlayingFlute_g19_c07.avi',
 'v_BalanceBeam_g03_c01.avi',
 'v_Billiards_g24_c05.avi',
 'v_GolfSwing_g17_c03.avi',
 'v_PizzaTossing_g11_c02.avi',
 'v_BlowDryHair_g23_c02.avi',
 'v_JavelinThrow_g03_c02

In [7]:
def get_UCF_classname(filename):
  return filename[filename.find('_') + 1:filename.find('_', 2)]

In [10]:
width, height, depth = 32, 32, 10
channel = 3

In [11]:
import cv2
import numpy as np

def video3d(filename, color=False, skip=True):
  cap = cv2.VideoCapture(filename)
  nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  if skip:
    frames = [x * nframe / depth for x in range(depth)]
  else:
    frames = [x for x in range(depth)]
  framearray = []

  for i in range(depth):
      cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])
      ret, frame = cap.read()
      frame = cv2.resize(frame, (height,width))
      if color:
          framearray.append(frame)
      else:
          framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

  cap.release()
  return np.array(framearray)

In [12]:
video_dir="/content/UCF101"
nclass=5
for filename in files:
  name = os.path.join(video_dir, filename)
  label = get_UCF_classname(filename)
  if label not in labellist:
    if len(labellist) >= nclass:
      continue
    labellist.append(label)
  labels.append(label)
  X.append(video3d(name, color=True, skip=True))

In [13]:
with open(os.path.join('classes.txt'), 'w') as fp:
  for i in range(len(labellist)):
      fp.write('{}\n'.format(labellist[i]))
for num, label in enumerate(labellist):
    for i in range(len(labels)):
        if label == labels[i]:
            labels[i] = num

In [14]:
labels

[0,
 1,
 2,
 3,
 4,
 2,
 0,
 2,
 0,
 4,
 0,
 2,
 2,
 1,
 1,
 2,
 4,
 2,
 1,
 0,
 1,
 1,
 4,
 2,
 3,
 4,
 1,
 0,
 4,
 1,
 2,
 0,
 3,
 2,
 4,
 2,
 3,
 1,
 4,
 2,
 4,
 1,
 3,
 0,
 2,
 1,
 4,
 3,
 3,
 0,
 0,
 2,
 1,
 0,
 0,
 1,
 0,
 3,
 2,
 3,
 4,
 0,
 2,
 1,
 2,
 3,
 0,
 4,
 1,
 0,
 1,
 1,
 0,
 2,
 2,
 2,
 1,
 0,
 3,
 2,
 4,
 1,
 0,
 2,
 1,
 3,
 2,
 4,
 0,
 0,
 1,
 3,
 1,
 0,
 2,
 3,
 4,
 4,
 4,
 1,
 3,
 3,
 1,
 2,
 2,
 0,
 2,
 3,
 2,
 2,
 2,
 3,
 4,
 4,
 2,
 3,
 4,
 4,
 0,
 4,
 1,
 1,
 0,
 3,
 4,
 4,
 1,
 4,
 1,
 1,
 2,
 2,
 1,
 0,
 0,
 0,
 4,
 0,
 3,
 2,
 1,
 1,
 1,
 0,
 2,
 4,
 1,
 1,
 1,
 0,
 1,
 4,
 2,
 3,
 0,
 4,
 4,
 1,
 3,
 0,
 0,
 1,
 2,
 0,
 3,
 3,
 2,
 2,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 3,
 2,
 2,
 3,
 0,
 0,
 1,
 4,
 2,
 2,
 0,
 3,
 3,
 1,
 3,
 0,
 3,
 3,
 0,
 1,
 3,
 4,
 0,
 4,
 3,
 0,
 3,
 0,
 2,
 2,
 3,
 3,
 2,
 3,
 2,
 0,
 1,
 2,
 3,
 0,
 0,
 4,
 4,
 3,
 1,
 0,
 3,
 3,
 1,
 1,
 2,
 4,
 1,
 4,
 2,
 1,
 4,
 1,
 3,
 3,
 0,
 3,
 0,
 4,
 1,
 0,
 4,
 0,


In [15]:
X = np.array(X).transpose((0, 2, 3, 4, 1))

In [20]:
#from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution3D, MaxPooling3D
from keras.layers import (Activation, Conv3D, Dense, Dropout, Flatten,MaxPooling3D)
from keras.utils import np_utils

In [21]:
Y = np_utils.to_categorical(labels,nclass)

X = X.astype('float32')
print('X_shape:{}\nY_shape:{}'.format(X.shape, Y.shape))

X_shape:(695, 32, 32, 3, 10)
Y_shape:(695, 5)


In [25]:
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(X.shape[1:]), padding='same'))
model.add(Activation('relu'))
model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(nclass, activation='softmax'))

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 32, 32, 3, 32)     8672      
                                                                 
 activation_4 (Activation)   (None, 32, 32, 3, 32)     0         
                                                                 
 conv3d_5 (Conv3D)           (None, 32, 32, 3, 32)     27680     
                                                                 
 activation_5 (Activation)   (None, 32, 32, 3, 32)     0         
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 11, 11, 1, 32)    0         
 3D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 11, 11, 1, 32)     0         
                                                      

In [27]:
 from sklearn.model_selection import train_test_split
 X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=4)




In [29]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=128,epochs=100, verbose=1)

Epoch 1/100
5/5 [==============================] - 14s 176ms/step - loss: 1.8086 - accuracy: 0.2230 - val_loss: 1.7253 - val_accuracy: 0.2086
Epoch 2/100
5/5 [==============================] - 0s 62ms/step - loss: 1.8450 - accuracy: 0.2212 - val_loss: 1.6228 - val_accuracy: 0.2086
Epoch 3/100
5/5 [==============================] - 0s 56ms/step - loss: 1.8362 - accuracy: 0.1655 - val_loss: 1.5981 - val_accuracy: 0.2014
Epoch 4/100
5/5 [==============================] - 0s 55ms/step - loss: 1.8236 - accuracy: 0.1888 - val_loss: 1.5903 - val_accuracy: 0.2806
Epoch 5/100
5/5 [==============================] - 0s 54ms/step - loss: 1.7752 - accuracy: 0.1906 - val_loss: 1.6283 - val_accuracy: 0.2086
Epoch 6/100
5/5 [==============================] - 0s 54ms/step - loss: 1.7927 - accuracy: 0.1996 - val_loss: 1.6418 - val_accuracy: 0.2086
Epoch 7/100
5/5 [==============================] - 0s 55ms/step - loss: 1.8009 - accuracy: 0.1978 - val_loss: 1.5965 - val_accuracy: 0.2014
Epoch 8/100
5/5 [=

In [30]:
model.evaluate(X_test, Y_test, verbose=0)

[0.46398699283599854, 0.8489208817481995]